In [1]:
from autogen import ConversableAgent
from openai import OpenAI
from pathlib import Path

import LLM_configs
import Prompt_dics
from LLM_configs import *
from Agents import *
from tools import *

In [2]:
# 加载 .env 文件
load_dotenv()

True

## 文件提取

In [3]:
text = file_extraction("test files/巨量均衡 2.docx")

In [4]:
PPT = file_extraction("test files/上海象上投资.pptx")

## Agent设计

In [5]:
quant_dict = {
    'quant_idea':[],
    'factor_composition':[],
    'factor_explanation':[],
    'model_explanation':[],
    'factor_update':[],
    'strategy_execution':[]
}

In [6]:
market_neutral = fill_placeholders(['中证500指增', '股指期货','0%'], Prompt_dics.Other_prompt_dic['中性多头/对冲端'])

In [7]:
market_neutral

'管理人的多头端是 中证500指增， 使用 股指期货进行对冲，留0%的风险敞口.'

In [8]:
def report_writing(weak_config, strong_config,prompts,component_dict, requirements, updating):
    extraction_agent = extraction_assistant(weak_config)
    
    integrate_agent = integrate_assistant(strong_config,requirements)
    
    for prompt, component in zip(prompts, list(component_dict.keys())):
        reply = extraction_agent.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": prompt, "role": "user"}])
        component_dict[component].append(reply)
    
    component_dict['hedging tools'] = market_neutral
    
    final_report = integrate_agent.generate_reply(messages=[{"content": str(component_dict), "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整，长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    non_updating_report = final_report
    
    if updating:
        old_report_relevant_part = input("请把上一篇报告的配置逻辑部分复制到此处")
        final_report = integrate_agent.generate_reply(messages=[{"content": "旧报告: "+old_report_relevant_part+"新报告:"+str(final_report), "role": "system"},
                                                                {"content": "你是一个报告审查和撰写专员，现在你有一篇旧报告和一篇新报告，你要做的是在旧报告的基础上，判断新报告中的增量信息，并将新报告中的增量信息与旧报告中的现有信息相结合，完成一篇更新的报告，如果两个报告中对某一个地方有冲突，则以新报告为准。", "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整.长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    
    cost_list = [extraction_agent.get_total_usage(),integrate_agent.get_total_usage()]
    
    return final_report,non_updating_report, component_dict, cost_list

In [9]:
Ali_report,first_Ali_report, Quant_component_dic, cost_list = report_writing(LLM_configs.Qwen_weak_llm_config, LLM_configs.Qwen_strong_llm_config, Prompt_dics.Strategy_prompt_dic['Simple Quant Strategy'],quant_dict, Prompt_dics.Strategy_requirements_dic['Simple Quant Strategy'],True)


>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...


In [10]:
print(Ali_report)

管理人的量化策略整体思路侧重于逻辑驱动与因子的可解释性，约60%-70%的精力集中于因子研究，20%专注于模型构建，彰显了对高质量、逻辑性强因子的高度重视。相较于过去以线性模型为主的股票ALPHA策略，当前策略在保持原有基础上，自2017年起引入并深入研究机器学习模型，形成了更为多元化的量化体系。策略底层因子构成约为60%基本面量化与40%AI量化，这一比例体现了管理人在传统量化与现代技术之间的平衡布局。基本面量化因子覆盖了50%的比例，利用财报、分析师预期、产业和商品数据，每日更新以筛选出截面上排名靠前的一篮子个股。此外，管理人凭借早期公募积累的行业研究资源，对10个行业实施单独建模，先进行全行业股票的alpha预测，随后通过单行业模型进行局部alpha优化。AI量化方面，占因子组成的40%，主要利用纯量价数据，分为两大模型类别：一是基于开源框架的端到端个股收益率预测，人工精细处理样本切割和特征提取；二是创新性地将深度学习模型的倒数第二层替换为树模型，以增强模型的适应性和解释力。模型训练后的评估严格，需经过不同时间样本的检验，确保模型的稳健性与外推能力，且模型参数调整频率低，每年定期进行，但在遭遇显著回撤时会及时复审并调整。

因子挖掘与处理上，基本面因子遵循“先逻辑后量化”的原则，确保因子背后的逻辑清晰可循，而量价因子虽应用复杂模型，同样重视逻辑性和实践验证。模型构建上，偏好线性模型如线性回归来处理基本面因子，以维护逻辑的直接性和模型的泛化性能，同时融入LGBM等非线性模型以增加模型的灵活性。对于量价因子，则采用GRU、RNN等高级非线性模型，以捕捉市场的非线性动态。尽管新报告未详述因子入库存储的具体条件，但可以推测管理人应有一套严谨的因子评估与纳入机制。

执行层面，策略综合运用基本面与量价因子，通过模型融合实施，尽管具体持股数量、指数内占比、行业及风格偏离度、年化双边换手率等操作细节未予明确，但策略迭代后已提升了对市场权重行情的敏感度，有效控制了年后回撤至4%，显示出策略持续优化的潜力。管理人计划进一步迭代策略，预计月底完成，预示着策略效能的进一步提升。在风险管理上，管理人采取中证500指增作为多头策略，并利用股指期货进行对冲，严格控制风险敞口为0%，体现了其严谨的风险管理哲学。风格约束方面，不仅采用传统Barra模型，还融入了团队自定义的十余个风险因子，虽

In [11]:
print(first_Ali_report)

该公司的量化策略核心思路侧重于逻辑驱动与因子的可解释性，约60%-70%的精力投入因子研究，20%聚焦于模型构建，显示了对高质量、逻辑性强因子的重视。策略因子组成上，基本面因子占比约60%，涵盖财务指标、分析师数据及另类因子；量价因子占比40%，涉及高频、中期和长期量价特征，另类因子则包括产业链上下游及舆情数据，因子池总量约5000-6000个，实际应用数百个。因子挖掘与处理上，基本面因子遵循“先逻辑后量化”原则，确保投资逻辑的明确性，而量价因子虽采用复杂模型，仍重视逻辑性和有效性。模型构建方面，公司偏好线性模型如线性回归用于基本面分析，以保证逻辑清晰和泛化能力，同时利用LGBM等简单非线性模型。量价分析则采用更复杂的非线性模型，包括GRU、RNN等，旨在捕捉市场非线性动态。因子入库存储条件未明确提及。执行层面，策略结合基本面与量价因子，通过不同模型融合实施，但具体持股数量、指数内占比、行业及风格偏离度、年化双边换手率等细节未予透露。管理人采取中证500指增作为多头端，并利用股指期货进行对冲，维持0%的风险敞口，体现了其对风险控制的严谨态度。


In [12]:
cost_list

[{'total_cost': 0.26224,
  'qwen-plus': {'cost': 0.26224,
   'prompt_tokens': 60337,
   'completion_tokens': 1741,
   'total_tokens': 62078}},
 {'total_cost': 0.27496000000000004,
  'qwen-max': {'cost': 0.27496000000000004,
   'prompt_tokens': 3688,
   'completion_tokens': 1062,
   'total_tokens': 4750}}]

In [13]:
cost_list

[{'total_cost': 0.26224,
  'qwen-plus': {'cost': 0.26224,
   'prompt_tokens': 60337,
   'completion_tokens': 1741,
   'total_tokens': 62078}},
 {'total_cost': 0.27496000000000004,
  'qwen-max': {'cost': 0.27496000000000004,
   'prompt_tokens': 3688,
   'completion_tokens': 1062,
   'total_tokens': 4750}}]

In [14]:
Quant_component_dic

{'quant_idea': ['该公司的量化策略整体上是偏重逻辑的，对因子的可解释性有很高的要求。具体体现在以下几个方面：\n\n- 在因子模型构建上，公司大约花费60%-70%的精力在因子研究上，而20%的精力则放在模型上。这表明公司更侧重于因子的质量和逻辑性。\n\n- 因子覆盖范围广泛，包括基本面因子、量价因子和另类因子。其中，基本面因子的研究深度较大，占据了较大的比重。\n\n- 在模型使用上，公司更倾向于使用经典模型和简单模型，如线性模型和LGBM（Light Gradient Boosting Machine）。这反映出公司重视模型的泛化能力和逻辑性。\n\n- 公司的基本面因子构建遵循“先逻辑后量化”的原则，即先确定投资逻辑，再通过量化手段表达这一逻辑。这进一步强调了策略的逻辑性和可解释性。\n\n- 量价因子方面，虽然也会使用一些复杂的模型，但整体上仍然注重模型的逻辑性和有效性。\n\n综上所述，该公司的量化策略更偏向于逻辑驱动型，注重因子的逻辑性和可解释性，而不是仅仅依赖于数据挖掘和机器学习算法。'],
 'factor_composition': ['管理人的策略因子组成主要包括以下几个方面：\n\n- **基本面因子**：\n  - 占比：大约60%的收益贡献。\n  - 构成：\n    - 财务指标和公告相关的因子，约占基本面因子的40%。\n    - 基于分析师的数据，同样约占基本面因子的40%。\n    - 另类和媒体因子，约占基本面因子的20%。\n\n- **量价因子**：\n  - 占比：大约40%的收益贡献。\n  - 构成：\n    - 高频、中期和长期量价因子。\n\n- **另类因子**：\n  - 包括产业链上下游数据和舆情因子等。\n\n因子池总数约为5000到6000个，但每期实际使用的因子数量大约为几百个。'],
 'factor_explanation': ['管理人详细解释了几种不同类型的因子：\n\n### 基本面因子\n- **构成**：\n  - 财务指标和公告相关的因子（40%）\n  - 基于分析师的数据（40%）\n  - 另类和媒体因子（20%）\n- **数据来源**：\n  - 上市公司的财报预告、快报等公告\n  - 分析师的报告和预期\n  - 产业链上下游数据\n  - 舆情数据

In [ ]:
def report_updating(weak_config, strong_config,prompts,component_dict, requirements):
    